In [5]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [6]:

from __future__ import absolute_import
from __future__ import print_function
import pylab as pl
import matplotlib.cm as cm
import itertools
import numpy as np
import csv
np.random.seed(1337) # for reproducibility

from keras.layers.noise import GaussianNoise
import keras.models as models
from keras.layers.core import Layer, Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

from keras import backend as K

import cv2
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [7]:
#load datasets
DATASETS_PATH = "./MontgomerySet/CXR_png/"

PREFIX = "MCUCXR_"
SUFFICES = [ "_0.png","_1.png"]
def get_image_filename(index):
    num_zeros = 4-len(str(index))
    if(index < 104):
        img_path = PREFIX + "0"*num_zeros + str(index) + SUFFICES[0]
    else:
        img_path = PREFIX + "0"*num_zeros + str(index) + SUFFICES[1]
    return (DATASETS_PATH + img_path)

In [8]:
get_image_filename(1)

'./MontgomerySet/CXR_png/MCUCXR_0001_0.png'

In [9]:
get_image_filename(104)

'./MontgomerySet/CXR_png/MCUCXR_0104_1.png'

In [11]:
CAPTIONS_PATH = "./MontgomerySet/ClinicalReadings/"

caption_list = os.listdir(CAPTIONS_PATH)

captions =[]
for fname in caption_list:
    with open(os.path.join(CAPTIONS_PATH,fname)) as f:
            lines = f.read().splitlines(True)
            caption =lines[-1]
            captions.append(caption[:-1])
    f.close()
captions = np.asarray(captions)
pd.DataFrame(captions).to_csv("./MontgomerySet/Captions.csv")

In [19]:
BASE_PATH = "./MontgomerySet/"

reader = csv.reader(open(BASE_PATH + 'Captions.csv'))
X =[]
Y =[]

for i,row in enumerate(reader):
    if len(row) > 0:
        filename = get_image_filename(i)
        img_file = cv2.imread(filename)
        if img_file is not None:
            img_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2RGB)
            img_arr = np.asarray(img_file)
            X.append(img_arr)
            Y.append(row[1])
        else:
            print("No file found", filename)

No file found ./MontgomerySet/CXR_png/MCUCXR_0000_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0007_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0009_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0010_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0012_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0014_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0018_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0025_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0032_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0033_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0034_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0036_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0037_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0039_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0050_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0065_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_0066_0.png
No file found ./MontgomerySet/CXR_png/MCUCXR_006

In [ ]:
X = np.asarray(X)
y =np.asarray(Y)

In [24]:
def categorize_images(folder):
    labels =np.zeros((138,), dtype =int)
    for i,f in enumerate(os.listdir(folder)):
        labels[i] = 0
        if i >= 104:
            labels[i] = 1
    return labels

In [26]:
labels = categorize_images(DATASETS_PATH)
labels.shape

(138,)

In [ ]:
#caption vectorization
token_index ={}

for sample in y:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
max_length = 100
results = np.zeros(shape=(len(y),max_length, max(token_index.values())+ 1))

for i, sample in enumerate(y):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i,j,index] =1

In [30]:
def get_data(folder,ys):
    X = []
    y = ys

    for l in y:
        if l==1:
            end_file = str(l)+".png"
        else:
            end_file = str(l)+".png"
        for image_filename in os.listdir(folder):
                img_file = cv2.imread(folder  + image_filename)
                if img_file is not None:
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
    X = np.asarray(X)
    return X,y

In [ ]:
FOLDER ="./MontgomerySet/CXR_png/"
get_data(FOLDER,labels)